In [3]:
!pip install -r requirements.txt

In [5]:
import os

import networkx as nx
import pandas as pd
import graphistry

In [ ]:
# Environment variable setup
GRAPHISTRY_USER = os.getenv("GRAPHISTRY_USERNAME")
GRAPHISTRY_PASSWORD = os.getenv("GRAPHISTRY_PASSWORD")

# Part 2: Quantitative Networks: Social Network Analysis and Network Science

## Visualizing Networks with Graphistry

Throughout this part of the course we will be using `pygraphistry` and [Graphistry Hub](https://hub.graphistry.com/) [https://hub.graphistry.com/](https://hub.graphistry.com/) to visualize networks. Both are free for personal use and are powerful for visualizing networks large and small.

You can [signup](https://hub.graphistry.com/accounts/signup/) for a Graphistry account at [https://hub.graphistry.com/accounts/signup/](https://hub.graphistry.com/accounts/signup/) with your Github or Google account. Retain and use the username and password in 

In [ ]:
graphistry.register(
    api=3,
    username=GRAPHISTRY_USERNAME,
    password=GRAPHISTRY_PASSWORD,
)